In [1]:
import tensorflow as tf
import data_util
import cv2
import matplotlib.pyplot as plt
import resnet_V2
from tensorflow.keras.utils import plot_model
from tensorflow import keras

In [2]:
tf.random.set_seed(42)

## 데이터 불러오기

In [3]:
ds = data_util.get_cu_dataset(train_type='landmark', batch_size=16)

In [4]:
ds_size = len(list(ds))
ds_size

478

In [5]:
train_size = int(0.7 * ds_size)
val_size = int(0.15 * ds_size)
test_size = int(0.15 * ds_size)

train_ds = ds.take(train_size)
test_ds = ds.skip(train_size)
val_ds = test_ds.skip(val_size)
test_ds = test_ds.take(test_size)

In [6]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(16, 256, 256, 3)
(16, 16)


In [9]:
ae = resnet_V2.ResNetAE()
ae.load_weights('./trained_ckpt/model_5.ckpt')

In [10]:
en = ae.encoder

In [13]:
resnet = keras.models.Sequential()
resnet.add(keras.layers.Input(shape=(256, 256, 3)))
resnet.add(en)
resnet.add(keras.layers.Flatten())
resnet.add(keras.layers.Dense(16))
resnet.compile(loss=keras.losses.mse,
              optimizer='adam')

In [ ]:
ckpt_path = './ckpt/pre_regression'
callbacks_list = [
    # accuracy 기준 가장 높은 모델의 weight 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = ckpt_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # EarlyStopping
    # tf.keras.callbacks.EarlyStopping(
    #     monitor='val_loss',
    #     mode='min',
    #     verbose=1, 
    #     patience=20
    # )
]

hist = resnet.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=callbacks_list)   # loss 기록